In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import random
random.seed(500)
df = pd.read_csv(r"/home/natant/Thesis-plmbind/Testing_ground/info.csv")

In [3]:
other_families = list(df["DBD"][[i not in ["C2H2 ZF", "Homeodomain", "bHLH"] for i in df["DBD"]]].unique())

In [4]:
All_ZFs = df["HGNC symbol"][df["DBD"] == "C2H2 ZF"].to_list()
random.shuffle(All_ZFs)
ZNF_test = All_ZFs[:50]
ZNF_val = All_ZFs[50:100]
ZNF_train = All_ZFs[100:]

In [5]:
All_homeo = df["HGNC symbol"][df["DBD"] == "Homeodomain"].to_list()
random.shuffle(All_homeo)
Homeo_test = All_homeo[:5]
Homeo_val = All_homeo[5:10]
Homeo_train = All_homeo[10:]

In [6]:
All_bHLH = df["HGNC symbol"][df["DBD"] == "bHLH"].to_list()
random.shuffle(All_bHLH)
bHLH_test = All_bHLH[:5]
bHLH_val = All_bHLH[5:10]
bHLH_train = All_bHLH[10:]

In [7]:
random.shuffle(other_families)
test_families = other_families[:5]
val_families = other_families[5:10]
train_families = other_families[10:]

In [8]:
rest_test = df["HGNC symbol"][[i in test_families for i in df["DBD"]]].to_list()
rest_val = df["HGNC symbol"][[i in val_families for i in df["DBD"]]].to_list()
rest_train = df["HGNC symbol"][[i in train_families for i in df["DBD"]]].to_list()

In [9]:
# sanity check!!
len(rest_test + rest_val + rest_train + bHLH_test + bHLH_val + bHLH_train + Homeo_test + Homeo_val + Homeo_train + ZNF_test + ZNF_val + ZNF_train)

855

In [10]:
train_TFs = ZNF_train + Homeo_train + bHLH_train + rest_train
test_TFs = ZNF_test + Homeo_test + bHLH_test + rest_test
val_TFs = ZNF_val + Homeo_val + bHLH_val + rest_val

In [11]:
len(val_TFs)

104

In [12]:
len(test_TFs)

94

In [13]:
len(train_TFs)

657

In [14]:
source = ["C2H2 ZF", "C2H2 ZF", "C2H2 ZF", "Homeodomain", "Homeodomain", "Homeodomain", "bHLH", "bHLH", "bHLH"] + test_families + val_families + train_families
target = ["Train", "Val", "Test", "Train", "Val", "Test","Train", "Val", "Test"] + ["Test"]*len(test_families) + ["Val"]*len(val_families) + ["Train"]*len(train_families)
value = [len(ZNF_train), len(ZNF_val), len(ZNF_test), len(Homeo_train), len(Homeo_val), len(Homeo_test), len(bHLH_train), len(bHLH_val), len(bHLH_test)] + [len(df["HGNC symbol"][df["DBD"] == i]) for i in test_families] + [len(df["HGNC symbol"][df["DBD"] == i]) for i in val_families] + [len(df["HGNC symbol"][df["DBD"] == i]) for i in train_families]
sankey_df = pd.DataFrame({"source": source, "target": target, "value": value})

In [15]:
from ipysankeywidget import SankeyWidget
from ipywidgets import Layout
layout = Layout(width="1000", height="1000")
SankeyWidget(links=sankey_df.to_dict('records'), layout=layout)

SankeyWidget(layout=Layout(height='1000', width='1000'), links=[{'source': 'C2H2 ZF', 'target': 'Train', 'valu…

In [16]:
source = ["C2H2 ZF", "C2H2 ZF", "C2H2 ZF", "Homeodomain", "Homeodomain", "Homeodomain", "bHLH", "bHLH", "bHLH"] + ["test_families", "val_families", "train_families"]
target = ["Train", "Val", "Test", "Train", "Val", "Test","Train", "Val", "Test"] + ["Test"] + ["Val"] + ["Train"]
value = [len(ZNF_train), len(ZNF_val), len(ZNF_test), len(Homeo_train), len(Homeo_val), len(Homeo_test), len(bHLH_train), len(bHLH_val), len(bHLH_test)] + [len(rest_test), len(rest_val), len(rest_train)]
sankey_df = pd.DataFrame({"source": source, "target": target, "value": value})

In [17]:
from ipysankeywidget import SankeyWidget
from ipywidgets import Layout
SankeyWidget(links=sankey_df.to_dict('records'))

SankeyWidget(links=[{'source': 'C2H2 ZF', 'target': 'Train', 'value': 367}, {'source': 'C2H2 ZF', 'target': 'V…

In [18]:
from floweaver import *
nodes = {
    'Families': ProcessGroup(["C2H2 ZF", "Homeodomain", "bHLH", "test_families", "val_families", "train_families"]),
    'Targets': ProcessGroup(['Train', 'Val', 'Test']),
}

ordering = [
    ['Families'],       # put "farms" on the left...
    ['Targets'],   # ... and "customers" on the right.
]

bundles = [
    Bundle('Families', 'Targets'),
]

families_with_other = Partition.Simple('big 3', [
    'C2H2 ZF',  # the groups within the partition can be a single id...
    'Homeodomain',
    'bHLH',
    ('other', ['test_families', 'val_families', 'train_families']),   # ... or a group
])

# This is another partition.
customers_by_name = Partition.Simple('process', [
    'James', 'Mary', 'Fred', 'Susan'
])

# Update the ProcessGroup nodes to use the partitions
nodes['Families'].partition = families_with_other
#nodes['customers'].partition = customers_by_name

sdd = SankeyDefinition(nodes, bundles, ordering)
weave(sdd, sankey_df).to_widget()


KeyError: 'big 3'